In [2]:
import os 
import pandas as pd

import ast

from tqdm import tqdm, tqdm_notebook
tqdm.pandas(tqdm_notebook)

In [3]:
loved_pages_df = pd.read_csv(
    '/Users/suganyasivaskantharajah/code/govuk_ab_analysis/loved_pages.csv.gz',
    usecols=['pagePath'])

In [5]:
# dedupe the pagePaths here just in case
loved_page_paths = list(set(loved_pages_df['pagePath'].tolist()))

In [9]:
len(loved_page_paths)

5496

In [7]:
REQUIRED_COLUMNS = ["Occurrences", "ABVariant", "Page_Event_List",
                    "Page_List",  "Event_cat_act_agg"]

In [8]:
df_2019_02_14 = pd.read_csv(
    '../data/processed_journey/taxon_ab_2019-02-14.csv.gz', sep='\t', 
    usecols=REQUIRED_COLUMNS)

In [10]:
df_2019_02_14['Page_List'] = df_2019_02_14['Page_List'].progress_apply(ast.literal_eval)

100%|██████████| 1386042/1386042 [00:27<00:00, 50069.42it/s]


In [17]:
loved_page_paths_set = set(loved_page_paths)

In [18]:
def is_loved_journey(page_list):
#     we dedupe each pagelist here
    return any(
        [page in loved_page_paths_set for page in list(set(page_list))])

In [19]:
df_2019_02_14['is_loved_journey'] = df_2019_02_14['Page_List'].progress_apply(is_loved_journey)


100%|██████████| 1386042/1386042 [00:04<00:00, 298312.33it/s]

In [20]:
df_2019_02_14.groupby('is_loved_journey').sum()

,Occurrences
is_loved_journey,
False,1022720
True,2489234


In [21]:
df_2019_02_14[df_2019_02_14['is_loved_journey']==False].to_csv(
    '../data/processed_journey/unloved_2019_02_14.csv.gz', 
    sep="\t", compression="gzip", index=False)

In [22]:
df_2019_02_14[df_2019_02_14['is_loved_journey']==True].to_csv(
    '../data/processed_journey/loved_2019_02_14.csv.gz', 
    sep="\t", compression="gzip", index=False)

In [24]:
df_2019_02_15 = pd.read_csv(
    '../data/processed_journey/taxon_ab_2019-02-15.csv.gz', sep='\t', 
    usecols=REQUIRED_COLUMNS)

In [25]:
df_2019_02_15['Page_List'] = df_2019_02_15['Page_List'].progress_apply(
    ast.literal_eval)


 51%|█████     | 657758/1294250 [00:14<00:14, 43428.19it/s]


100%|██████████| 1294250/1294250 [00:27<00:00, 46675.61it/s]

In [26]:
df_2019_02_15['is_loved_journey'] = df_2019_02_15['Page_List'].progress_apply(is_loved_journey)


100%|██████████| 1294250/1294250 [00:04<00:00, 286515.59it/s]

In [33]:
df_2019_02_15.groupby('is_loved_journey').sum()

                  Occurrences
is_loved_journey             
False                  942642
True                  2376548


In [34]:
df_2019_02_15[df_2019_02_15['is_loved_journey']==False].to_csv(
    '../data/processed_journey/unloved_2019_02_15.csv.gz', 
    sep="\t", compression="gzip", index=False)

In [31]:
df_2019_02_15[df_2019_02_15['is_loved_journey']==True].to_csv(
    '../data/processed_journey/loved_2019_02_15.csv.gz', 
    sep="\t", compression="gzip", index=False)

In [41]:
def split_daily_data(file_prefix):
    df = pd.read_csv(
        f'../data/processed_journey/taxon_ab_{file_prefix}.csv.gz', sep='\t', 
        usecols=REQUIRED_COLUMNS)
    df['Page_List'] = df['Page_List'].progress_apply(
        ast.literal_eval)
    df['is_loved_journey'] = df['Page_List'].progress_apply(is_loved_journey)
    print(df.groupby('is_loved_journey').sum())
    df[df['is_loved_journey']==False].to_csv(
        f'../data/processed_journey/unloved_{file_prefix}.csv.gz', 
        sep="\t", compression="gzip", index=False)
    df[df['is_loved_journey']==True].to_csv(
        f'../data/processed_journey/loved_{file_prefix}.csv.gz', 
        sep="\t", compression="gzip", index=False)

In [42]:
split_daily_data('2019-02-16')


 81%|████████  | 653754/810323 [00:29<00:03, 49332.46it/s]


100%|██████████| 810323/810323 [00:02<00:00, 291799.77it/s]

                  Occurrences
is_loved_journey             
False                  499900
True                  1544902


In [43]:
split_daily_data('2019-02-17')


 83%|████████▎ | 704679/849131 [00:13<00:02, 52938.43it/s]


100%|██████████| 849131/849131 [00:02<00:00, 301213.05it/s]

                  Occurrences
is_loved_journey             
False                  506690
True                  1565376


In [44]:
split_daily_data('2019-02-18')


 42%|████▏     | 678661/1626133 [00:15<00:17, 54234.37it/s]


 83%|████████▎ | 1346671/1626133 [00:29<00:05, 50033.70it/s]


100%|██████████| 1626133/1626133 [00:05<00:00, 286995.74it/s]

                  Occurrences
is_loved_journey             
False                 1170400
True                  2962617


In [40]:
'/apply-to-come-to-the-uk' in loved_page_paths

True